# C++ Review - Part 3 - Classes, Polymorphism, Collections

## Classes

- Represent structured data
- Bind data to functions
  - Curry example: full arguments, then in a class
- Encapsulate mutable state
  - State is tricky. Classes give us a mechanism to control how and when state can change, making it easier to reason about our code.
  - Example: data structures
  
Object-oriented vs functional: what matches real life better? Which is easier to reason about?

House example in the chapter: what functions make sense to put on the `House` class? Does it make sense to put `accessTaxes` on the `House` class? 

While the language gives you the ability to use a class in many ways, not all those ways make sense. Learn to identify the patterns that work best. Example: `goto`, `for`, and `while`. 

### Structured Data

In [1]:
#include <string>
#include <vector>

#include <iostream>
#include <fstream>
#include <sstream>

using namespace std;

In [2]:
class House {
    private:
    int sqFootage;
    string address;
    string zipCode;
    public:
    House(int sqFootage, string address, string zipCode) : sqFootage(sqFootage), address(address), zipCode(zipCode) {}
    
    int getSqFootage() const {
        return sqFootage;
    }
    string getAddress() const {
        return address;
    }
    string getZipCode() const {
        return zipCode;
    }
    string str() const {
        stringstream ss;
        ss << address << " " << zipCode << " " << sqFootage << "sqFt";
        return ss.str();
    }
}

What is inside the class `House`?

`private` vs `public`?

What is a constructor?

What does `const` mean? Why are the "getters" `const`?

What is a common file structure for C++ classes? What typically goes in a `.h` file? What about a `.cpp` file?

<div class="big center">🏠</div>

Scope resolution operation.

`this` pointer

Assignment statements vs initializers.

Setters



####  `house.h`

```c++
#include <string>
using namespace std;

class House {
    private:
    int sqFootage;
    string address;
    string zipCode;
    public:
    House(int, string, string);
    int getSqFootage() const;
    string getAddress() const;
    string getZipCode() const;
    string str() const;
};
```

#### `house.cpp`
```c++
#include <iostream>
#include <sstream>

#include "house.h"

using namespace std;

House::House(int sqFootage, string address, string zipCode) 
    : sqFootage(sqFootage), address(address), zipCode(zipCode) 
    {
        // Nothing needed here
    }
    
int House::getSqFootage() const {
    return sqFootage;
}
string House::getAddress() const {
    return address;
}
string House::getZipCode() const {
    return zipCode;
}
string House::str() const {
    stringstream ss;
    ss << address << " " << zipCode << " " << sqFootage << "sqFt";
    return ss.str();
}

int main() {
    House h1 = House(700, "123 St", "12345");
    cout << h1.str() << endl;
}
```

What happens when you include `house.h` twice?

<div class="big center">🏚</div>

What can you do to fix it?

Why not:
```c++
class House {
    public:
    int sqFootage;
    string address;
    string zipCode;
}
```

### Structured behavior

In [3]:
#include <set>

In [4]:
float accessTaxes(House const& house, set<string> freeZipCodes) {
    // Is the house zip code included in the free zip codes?
    if (freeZipCodes.find(house.getZipCode()) != freeZipCodes.end()) {
        // This zip code doesn't pay taxes
        return 0.0;
    } else {
        // This zip code does
        return 0.08 * house.getSqFootage();
    }
}

Why **`const&`** in `House const& house`?

What abstraction does this function represent? What are the rules that this abstractions implies?

Should the return value be dependent on the input? 

Should a `house` change when you assess taxes on it?

Should the set of free zip codes change when you assess taxes on a house?

How can we make our function more true to the previous assertion?

Let's assess taxes on some houses. The concept of "assessing taxes" is an abstraction. What assumptions do we have about that abstraction?

In [5]:
auto h1 = House(700, "street 1", "12345");
auto h2 = House(4000, "street 2", "54321");

In [6]:
set<string> freeCodes;
freeCodes.insert("12345");

In [7]:
accessTaxes(h1, freeCodes)

0.00000f

In [8]:
accessTaxes(h2, freeCodes)

320.000f

Is there anything in the code that guarantees that `h1` and `h2` are assessed using the same zip code rules?

In [9]:
class TaxAssessor {
    private:
    set<string> freeZipCodes;
    public:
    TaxAssessor(set<string> freeZipCodes): freeZipCodes(freeZipCodes) {}
    
    float assessTaxes(House const& house) {
        // Is the house zip code included in the free zip codes?
        if (freeZipCodes.find(house.getZipCode()) != freeZipCodes.end()) {
            // This zip code doesn't pay taxes
            return 0.0;
        } else {
            // This zip code does
            return 0.08 * house.getSqFootage();
        }
    }
}

In [11]:
TaxAssessor taxAssessor(freeCodes);

In [12]:
taxAssessor.assessTaxes(h1)

0.00000f

In [13]:
taxAssessor.assessTaxes(h2)

320.000f

What if we want to change the tax law and see what will happen?

In [15]:
set<string> noCodes;
TaxAssessor proposedTaxAssessor(noCodes);

In [16]:
proposedTaxAssessor.assessTaxes(h1)

56.0000f

In [17]:
proposedTaxAssessor.assessTaxes(h2)

320.000f

The class `TaxAssessor` allows us to predefine a set of common parameters that will be used across invocations of the method.

## Polymorphism

Polymorphism is about abstractions. It's the concept of defining an interface, or the qualities of an abstraction that are important in your context, and then building your code against that interface. 

```c++
class TaxAssessorInterface {
    virtual float assessTaxes(House const&) = 0;
};
```

```c++
float computeTotalTaxes(vector<House> const& houses, TaxAssessorInterface const& taxAssessor) {
    float total = 0;
    for (int i = 0; i < houses.size(); i++) {
        total += taxAssessor.assessTaxes(houses[i]);
    }
    return total;
}
```

The `computeTotalTaxes` class doesn't care how `taxAssessor` is implemented. It just knows that it can call `.assessTaxes()` with a `House` and get the taxes back. That's all it needs to know.

The business logic for how to compute a total (e.g. to sum up all values in list) is now *decoupled* from the business logic for how to assess taxes on an individual house. The logic for how to do either task is free to evolve without disturbing the other.

In [21]:
class TaxAssessorInterface {
    virtual float assessTaxes(House const&) = 0;
};

class ZipCodeTaxAssessor: public TaxAssessorInterface {
    private:
    set<string> freeZipCodes;
    public:
    ZipCodeTaxAssessor(set<string> freeZipCodes): freeZipCodes(freeZipCodes) {}
    
    float assessTaxes(House const& house) {
        // Is the house zip code included in the free zip codes?
        if (freeZipCodes.find(house.getZipCode()) != freeZipCodes.end()) {
            // This zip code doesn't pay taxes
            return 0.0;
        } else {
            // This zip code does
            return 0.08 * house.getSqFootage();
        }
    }
};

**WARNING**: if in order to understand the behavior of a piece of code, you have to understand the implementation of code in another file, you will suffer.

Avoid spreading the implementation of an object across multiple class definitions. Class inheritance in C++ will let you do this, but you shouldn't. 

> Everything you can do with inheritance can be done with composition, only better.

Use class inheritance in C++ as a means to implement polymorphism. Use only stateless abstract classes as base classes. 

There are exceptions to this principle, and we may see some of them in this class, but always go into a design decision very skeptical of inheritance as a possible solution.

## Principles

What are the necessary qualities of an abstraction in a given context?

Does your implementation manifest those qualities?

Does the abstraction and implementation guide you into doing the right thing?

**It's possible to use correctly** *vs* **It's impossible to use incorrectly**

Classes give you the ability to *bind* data to functions—i.e. to constrain a broad range of behavior into a specific use-case. 

Classes give you the ability to abstract the complexity of an implementation and present a simple interface. 

## Data Structures
State is gold. Don't mess it up.

We need ways of modifying and accessing the state of our program. 

Rules and patterns around how the state is modified and accessed provides sanity and security in a program.

Bank account: what rules apply to the state of your account?

House taxes: what if the set of zip codes could change in between invocations on `h1` and `h2`? Does that violate an assumption you relied on?

We write classes to carefully protect the data of our programs. 

These classes ensure that our code interacts with the state in the right ways, preserving the abstractions we have created around our state.

### If you have a collection of items...

If you have a collection of items, can you add another item to the collection?

Does it matter how long it takes to add an item?

Does it matter if the time it takes to add an item grows when the collection gets larger?

If you have a collection of items, can you remove items?

Does it matter how long it takes to remove an item?

If you have a collection of items, can the same item be present multiple times?

Can you tell whether a given item is present? 

If you have a collection of items, are the items ordered? 

Does the order depend on the order the items were included? 

Does the order depend on the properties of the items?

If you have a collection of items, how do you retrieve an item in the collection?